# Big Basket SKU Scraper

In [ ]:
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings("ignore")
from time import sleep
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from datetime import datetime
from selenium.webdriver.chrome.options import Options

In [ ]:
pin = pd.read_excel("bb_sheet.xlsx", sheet_name="pincode-city")
bb = pd.read_excel("bb_sheet.xlsx", sheet_name="bbcode1")

In [ ]:
pin

In [ ]:
bb

In [ ]:
a = pin.copy()
for i in range(len(bb)-1):
     a = pd.concat([a,pin])

In [ ]:
a

In [ ]:
b = bb.copy()
for i in range(len(pin)-1):
    b = pd.concat([b,bb])

In [ ]:
b

In [ ]:
b.sort_index(inplace=True)

In [ ]:
a["BB Code"] = list(b["BB Code"])
a["Product"] = list(b["Product"])

In [ ]:
a

In [ ]:
a.reset_index(inplace=True)

In [ ]:
a.drop("index",axis=1,inplace=True)

In [ ]:
bbcode = list(a["BB Code"])
pincode = list(a["Pincode"])
city = list(a["City"])

In [ ]:
path = r"C:\Users\Sanchit\Downloads\chromedriver.exe"
options = Options()
options.add_argument('--headless')
options.add_argument('--disable-gpu')  
options.add_argument('--no-sandbox')
options.add_argument('window-size=1920x1080')

In [ ]:
# price = []
# avail = []
# date = []
# time = []
# bbcodes = []
# star = []
# no_ratings = []
# no_reviews = []
count = 0
df = {"Date":[],"Pincode":[],"City":[],"BB Code":[],"Selling Price":[],"Availability":[]}

In [ ]:
for i in range(151,len(bbcode)):
    browser = webdriver.Chrome(executable_path=path,chrome_options=options)
    browser.get("https://www.bigbasket.com/pd/"+str(bbcode[i])+"/")
    sleep(3)

    pincode_button = browser.find_element_by_class_name('_1N37e')
    pincode_button.click()
    sleep(3)
        
    city_button = browser.find_element_by_class_name('_2xkur')
    city_button.click()
    sleep(1)
    city_name = browser.find_element_by_xpath("/html/body/div[1]/div/div[1]/div[2]/div/div/div[2]/div[1]/input")
    city_name.send_keys(city[i])
    df["City"].append(city[i])
    browser.find_element_by_class_name("oXkKp").click()
    
    pc = browser.find_element_by_xpath("//*[@id='modal']/div/div/div[2]/div[2]/input")
    pc.send_keys(str(pincode[i]))
    df["Pincode"].append(pincode[i])
    sleep(3)
        
    try:
        pincode_button = browser.find_element_by_xpath("//*[@id='modal']/div/div/div[2]/div[2]/div")
        pincode_button.click()
        sleep(1)
        browser.find_element_by_xpath("//*[@id='modal']/div/div/div[2]/form/button").click()
        sleep(1)
            
        try:
            sp = browser.find_element_by_class_name('IyLvo')
            sp = sp.text
            sp = sp.replace('Rs ','')
            sp = float(sp)
            df["Selling Price"].append(sp)               
            
        except NoSuchElementException:
            try:
                sp = browser.find_element_by_xpath('//*[@id="siteLayout"]/div/div/div[1]/section[1]/table/tbody/tr[1]/td[2]/text()')
                sp = sp.text
                sp = sp.replace('₹','')
                sp = sp.replace('Price: ','')
                sp = float(sp)
                df["Selling Price"].append(sp)
            except NoSuchElementException:
                df["Selling Price"].append(0)
                
            
        try:
            availability = browser.find_element_by_class_name("_36qqs")
            df["Availability"].append("Not Available")
        
        except NoSuchElementException:
            try:
                availability = browser.find_element_by_xpath('//*[@id="siteLayout"]/div/div/section[4]/div[2]/div/span')
                df["Availability"].append('Not Available')
            
            except NoSuchElementException:
                df["Availability"].append("Available")

        now = datetime.now()
        df["Date"].append(now)

    except ElementNotInteractableException:
        df["Selling Price"].append(0)
        df["Availability"].append("Address not deliverable")
        now = datetime.now()
        df["Date"].append(now)
    df["BB Code"].append(bbcode[i])
    count+=1
    print(count)
    browser.close()

In [ ]:
from collections import Counter
res = Counter(df["City"])

In [ ]:
res

In [ ]:
# df["BB Code"].pop(70)

In [ ]:
# res = Counter(df["BB Code"])
# res

In [ ]:
fin = pd.DataFrame.from_dict(df)

In [ ]:
fin

In [ ]:
from datetime import timedelta
fin["Date"] = pd.to_datetime(fin['Date'].astype(str)) + timedelta(hours=5,minutes=30)

In [ ]:
fin

In [ ]:
fin.to_excel("bb_fin_sheet1.xlsx")